In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from classification_plots import plot_confusion_matrix
import matplotlib.pyplot as plt
import os
from collections import Counter
%matplotlib notebook

Using TensorFlow backend.


In [2]:
import torch.nn as nn
import torch.nn.functional as F

# class Net(nn.Module):
#     def __init__(self,):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 9)
#         self.pool = nn.MaxPool2d(25, 17)
#         self.norm = nn.BatchNorm2d(32)
#         self.fc1 = nn.Linear(4608, 32)
#         self.dropout = nn.Dropout2d(p=0.2)
#         self.fc2 = nn.Linear(32, 32)
#         self.fc3 = nn.Linear(32,50)
#         self.softmax = nn.Softmax()

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.norm(x)
#         x = F.relu(x)
#         x = self.pool(x)
#         x = x.view(x.shape[0], -1)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.fc3(x)
#         x = self.softmax(x)
#         return x
class Net(nn.Module):
    def __init__(self,):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 9)
        self.conv2 = nn.Conv2d(32, 32, 5)
        self.norm = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(7, 7)
        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout2d(p=0.2)
#         self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,50)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.norm(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x
    
    
class BaseNet(nn.Module):
    def __init__(self,):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 50, 9)
        self.pool1 = nn.MaxPool2d((5,5), (4,4))
        self.conv2 = nn.Conv2d(50, 100, 7)
        self.pool2 = nn.MaxPool2d((5,5), (4,4))
        self.conv3 = nn.Conv2d(100, 100, 5)
        self.pool3 = nn.MaxPool2d((1,20))
        self.fc = nn.Linear(100, 50)
#         self.dropout = nn.Dropout2d(p=0.2)
#         self.fc2 = nn.Linear(32, 32)
#         self.fc3 = nn.Linear(32,50)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = F.relu(self.conv1(x))
#         print(x.shape)
        x = self.pool1(x)
#         print(x.shape)
        x = F.relu(self.conv2(x))
#         print(x.shape)
        x = self.pool2(x)
#         print(x.shape)
        x = F.relu(self.conv3(x)) 
#         print(x.shape)
        x = self.pool3(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        x = self.softmax(x)
        return x
    
    
net = Net()
basenet = BaseNet()
print(net)
print(basenet)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(9, 9), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (dropout): Dropout2d(p=0.2, inplace=False)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=50, bias=True)
  (softmax): Softmax(dim=None)
)
BaseNet(
  (conv1): Conv2d(1, 50, kernel_size=(9, 9), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(5, 5), stride=(4, 4), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(50, 100, kernel_size=(7, 7), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(5, 5), stride=(4, 4), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(100, 100, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=(1, 20), stride=(1,

In [3]:
# load data:
from PIL import Image
import os 
import numpy as np
from torch.utils.data import Dataset, DataLoader

class trainset(Dataset):
    def __init__(self):
        
        root = "./data2/"
        self.data_list = []
        self.label_list = []
        for root, dir, files in os.walk("./data2/"):
            for file in files:
                if file.find('.npy')!= -1:
                    self.data_list.append(os.path.join(root, file))
                    self.label_list.append(int(root[13:]))
        print(len(self.data_list), len(self.label_list))
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):

        path = self.data_list[index]
#         print(path)
#         data = []
#         label = []
        data = np.load(path,allow_pickle = True)
        data = np.expand_dims(data, axis=0)
#         data = np.asarray(Image.open(path))
        label= self.label_list[index] 
#         print(data.shape)
#         data = np.asarray(data)
        label = np.asarray(label)
        
        return data, label
dataloader = trainset()
data,label = dataloader.__getitem__(5)
# print(label.shape)
print(data.shape, label.shape)

2000 2000
(1, 128, 431) ()


In [4]:
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import time

# model = BaseNet()
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

validation_split = .2
random_seed= 42
shuffle_dataset = True
dataset_size = 2000
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

dataset  = trainset()
trainloader = torch.utils.data.DataLoader(dataset, batch_size=128, 
                                           sampler=train_sampler)
valloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                                sampler=valid_sampler)

val_history = []
val_loss_hist = []
train_history = []
train_loss_hist = []

for epoch in range(30):  # loop over the dataset multiple times
    
########## Validation ###########
    
    count = 0
    running_accuracy = 0
    running_loss = 0
    t1 = time.time()
    for i, data in enumerate(valloader, 0):
        count += 1
        inputs, labels = data
        inputs = inputs.float()
        outputs = model(inputs)
        print(outputs.shape, labels.shape)
        val_loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        acc_val = torch.eq(preds, labels).float().mean()
        running_accuracy += acc_val.item()
        running_loss += val_loss.item()
        
    running_accuracy /= count
    running_loss /= count
    val_history.append(running_accuracy)
    val_loss_hist.append(running_loss)
    t2 = time.time()
    print("===========Phase: Val============")
    print("Validation Time: {}".format(t2 - t1))
    print("Epoch: {}  val_loss: {}".format(epoch, running_loss))
    print("Epoch: {}  val_accuracy: {}".format(epoch, running_accuracy))
    
######### Training ###########   
    
    running_loss = 0.0
    count = 0
    training_accuracy = 0
    t1 = time.time()
    for i, data in enumerate(trainloader, 0):
        count += 1
        inputs, labels = data
        inputs = inputs.float()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        acc_train = torch.eq(preds, labels).float().mean()
        running_accuracy += acc_train.item()
        
    running_accuracy /= count
    running_loss /= count
    train_history.append(running_accuracy)
    train_loss_hist.append(running_loss)
    t2 = time.time()
    print("===========Phase: Train============") 
    print("Training Time: {}".format(t2 - t1))
    print("Epoch: {}  train_loss: {}".format(epoch, running_loss))
    print("Epoch: {}  train_accuracy: {}".format(epoch, running_accuracy))
    print()
        
# # #             torch.save(net, 'toy_model.pt')
        
    
print('Finished Training')


2000 2000


/root/anaconda3/envs/netdissect/lib/python3.6/site-packages/ipykernel_launcher.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([128, 50]) torch.Size([128])
torch.Size([128, 50]) torch.Size([128])


KeyboardInterrupt: 

In [22]:
torch.save(net, './model_v0.pt')

/root/anaconda3/envs/netdissect/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
